In [44]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Your portfolio: ticker symbols and shares owned
portfolio = {
    'AAPL': 10,
    'MSFT': 5,
    'TSLA': 3
}

# Fetch historical closing prices for last 3 months
tickers = list(portfolio.keys())
data = yf.download(tickers, period='3mo')['Close']

# Fetch S&P 500 index data for comparison
sp500 = yf.download('^GSPC', period='3mo')['Close']

# Calculate current value per stock and total portfolio value
latest_prices = data.iloc[-1]
values = latest_prices * pd.Series(portfolio)
total_value = values.sum()

print("Current Portfolio Value:")
print(values)
print(f"Total: ${total_value:.2f}\n")

# Calculate daily portfolio value (sum of all holdings)
daily_values = data.multiply(pd.Series(portfolio), axis=1).sum(axis=1)

# Calculate daily returns of portfolio and S&P 500
portfolio_returns = daily_values.pct_change().fillna(0)
sp500_returns = sp500.pct_change().fillna(0)

# Calculate cumulative returns
portfolio_cum_returns = (1 + portfolio_returns).cumprod() - 1
sp500_cum_returns = (1 + sp500_returns).cumprod() - 1

# Calculate annualized volatility of portfolio
trading_days = 252
volatility = portfolio_returns.std() * np.sqrt(trading_days)



[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  1 of 1 completed

Current Portfolio Value:
Ticker
AAPL    2068.600006
MSFT    2290.850067
TSLA    1031.460022
dtype: float64
Total: $5390.91



In [58]:
sp500_cum_returns.iloc[-1].item()*100

-1.208520741963548

In [64]:
# Print summary statistics
print(f"Portfolio Total Return (3 months): {portfolio_cum_returns[-1]*100:.2f}%")
print(f"Portfolio Annualized Volatility: {volatility*100:.2f}%")
print(f"S&P 500 Total Return (3 months): {sp500_cum_returns.iloc[-1]*100}%\n")

# Plot portfolio value and cumulative returns with two y-axes
fig = go.Figure()

# Portfolio value trace
fig.add_trace(go.Scatter(
    x=daily_values.index,
    y=daily_values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='blue')
))

# Portfolio cumulative returns trace (secondary y-axis)
fig.add_trace(go.Scatter(
    x=portfolio_cum_returns.index,
    y=portfolio_cum_returns,
    mode='lines',
    name='Portfolio Cumulative Return',
    yaxis='y2',
    line=dict(color='green')
))

# S&P 500 cumulative returns trace (secondary y-axis)
fig.add_trace(go.Scatter(
    x=sp500_cum_returns.index,
    y=sp500_cum_returns,
    mode='lines',
    name='S&P 500 Cumulative Return',
    yaxis='y2',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title='Portfolio Value and Cumulative Returns vs S&P 500',
    xaxis_title='Date',
    yaxis=dict(
        title='Portfolio Value (USD)',
        side='left'
    ),
    yaxis2=dict(
        title='Cumulative Return',
        overlaying='y',
        side='right',
        tickformat=',.0%',
        range=[
            # Extract the scalar minimum value from the Series returned by min(sp500_cum_returns)
            min(portfolio_cum_returns.min(), sp500_cum_returns.min().min()) * 1.1,
            max(portfolio_cum_returns.max(), sp500_cum_returns.max().max()) * 1.1
        ]
    ),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    height=600
)

fig.show()


Portfolio Total Return (3 months): -2.04%
Portfolio Annualized Volatility: 46.34%
S&P 500 Total Return (3 months): Ticker
^GSPC   -1.208521
Name: 2025-05-20 00:00:00, dtype: float64%



<ipython-input-64-bd2d7304d16b>:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

